# パーセプトロン

- **パーセプトロン**
    - 1957年ローゼンブラットにより考案されたアルゴリズム
    - 複数の信号を受け取り、一つの信号を出力する
        - 出力する信号は「流す／流さない（1 or 0）」の二値
        - 各入力信号には重要度があり、それを重みで表現する

![perceptron](./img/perceptron.png)

In [1]:
# 内積関数 dot を使えるようにする
using LinearAlgebra

# パーセプトロン
## 信号を受け取り、<入力信号> * <重み> + <バイアス> から出力（0 | 1）を計算するパーセプトロン関数を生成
make_perceptron(weight::Array{Float64,1}, bias::Float64) = begin
    # 入力信号x と 重みweight の内積に biasを加算した値が 0を超えているなら 1を返す
    return (x::Array{Int,1}) -> dot(x, weight) + bias > 0 ? 1 : 0
end

make_perceptron (generic function with 1 method)

In [2]:
"""
AND
0, 0 => 0
0, 1 => 0
1, 0 => 0
1, 1 => 1
"""

# ANDパーセプトロン
## 重み: 0.5, 0.5
## バイアス: -0.75
AND = make_perceptron([0.5, 0.5], -0.75)

for x in [[0, 0], [0, 1], [1, 0], [1, 1]]
    println("$(x) => $(AND(x))")
end

[0, 0] => 0
[0, 1] => 0
[1, 0] => 0
[1, 1] => 1


## パーセプトロンの数式の意味

パーセプトロンにおいて、$w_i$ は**入力信号の重要度**、$b$ は**ニューロンの発火のしやすさ**を表す

また、基本的に $\sum_{i=1}^n |w_i| = 1$ になり、$|b| \leqq 1$ となる

通常 $b < 0$ なので、$b$ の絶対値は**ニューロンの発火のしづらさ**を表す

この前提のもと、ANDパーセプトロンのパラメータの意味を考える

- $w_1, w_2$ ともに 0.5 であるため、2つの入力信号の重要度は変わらない
    - 言い換えれば、2つの入力値の入力順を変えても結果は変わらない
- $b$ は -0.75 であり、$ 0.25 - 1 $ に等しい
    - つまり、25％の確率で発火し、75％の確率で発火しないことを意味する

次に、実際のAND演算子を考えてみる

- 2つの入力値は、入力順が変わっても出力値は変わらない
    - $[0, 1] = [1, 0] = 0$
- 入力値の4つの組み合わせ（$[0, 0], [0, 1], [1, 0], [1, 1]$）のうち、出力値が1（発火）になるのは1つ（$[1, 1]$）のみ
    - 発火の確率は $1/4 = 0.25$ だから、25％

このように、入力信号の重要度と発火のしやすさだけを考えれば、ビット演算子のニューロンを組み立てることが可能である

In [3]:
"""
OR
0, 0 => 0
0, 1 => 1
1, 0 => 1
1, 1 => 1
"""

# ORパーセプトロン
## 重み: 0.5, 0.5 => 入力信号の重要度は変わらない
## バイアス: -0.25 => 25%の確率で発火しない（[0,0]のみ）, 75%の確率で発火（[0,1], [1,0], [1,1]）
OR = make_perceptron([0.5, 0.5], -0.25)

for x in [[0, 0], [0, 1], [1, 0], [1, 1]]
    println("$(x) => $(OR(x))")
end

[0, 0] => 0
[0, 1] => 1
[1, 0] => 1
[1, 1] => 1


In [4]:
"""
NAND: NOT AND
0, 0 => 1
0, 1 => 1
1, 0 => 1
1, 1 => 0
"""

# NANDパーセプトロン
## 重みとバイアスをそれぞれ符号反転すれば NOT 演算に等しくなる: NOT AND = -AND(x, b)
## 重み: -0.5, -0.5 => 入力信号の重要度は変わらない
## バイアス: 0.75 => 25%の確率で発火（[1,1]）
NAND = make_perceptron([-0.5, -0.5], 0.75)

for x in [[0, 0], [0, 1], [1, 0], [1, 1]]
    println("$(x) => $(NAND(x))")
end

[0, 0] => 1
[0, 1] => 1
[1, 0] => 1
[1, 1] => 0


In [5]:
"""
NOR: NOT OR
0, 0 => 1
0, 1 => 0
1, 0 => 0
1, 1 => 0
"""

# NORパーセプトロン
## 重みとバイアスをそれぞれ符号反転すれば NOT 演算に等しくなる: NOT OR = -OR(x, b)
## 重み: -0.5, -0.5 => 入力信号の重要度は変わらない
## バイアス: 0.25 => 75%の確率で発火（[0,1], [1,0], [1,1]）
NOR = make_perceptron([-0.5, -0.5], 0.25)

for x in [[0, 0], [0, 1], [1, 0], [1, 1]]
    println("$(x) => $(NOR(x))")
end

[0, 0] => 1
[0, 1] => 0
[1, 0] => 0
[1, 1] => 0


## 分類問題としてのパーセプトロン

パーセプトロンは、極めて簡単な式で分類問題を解くことができる

すなわち、入力信号として2つのビット値をとり、その出力値を 0 or 1 に分類するアルゴリズムであると言える

図示すると、以下のような直線で分類を行っていると考えることができる

![perceptron_and_or](./img/perceptron_and_or.png)


### パーセプトロンの限界
パーセプトロンによる分類は、直線を用いたものであるため、直線で区切ることができないデータを分類することはできない

ビット演算で言うと、XORなどがそれに当てはまる

下図のように、XORの出力値は直線で分類することができない

![perceptron_xor](./img/perceptron_xor.png)

## 多層パーセプトロン

### XOR回路図
XORは、正確に言えば「単一のパーセプトロンで分類することができない」だけであり、複数のパーセプトロンを重ねればこの問題を解くことができる

すなわちXORの分類問題は、**多層パーセプトロン**で解くことが可能である

そのためには、まずはXORゲートをAND, OR, NAND, NORゲートの組み合わせに分解する必要がある（下図）

![perceptron_xor](./img/perceptron_xor_gate.png)

In [6]:
"""
XOR: AND(NAND(x1,x2), OR(x1,x2))
0, 0 => 0
0, 1 => 1
1, 0 => 1
1, 1 => 0
"""

# XORパーセプトロン
## NAND, ORパーセプトロンを ANDパーセプトロンで結合する
XOR(x::Array{Int,1}) = AND([NAND(x), OR(x)])

for x in [[0, 0], [0, 1], [1, 0], [1, 1]]
    println("$(x) => $(XOR(x))")
end

[0, 0] => 0
[0, 1] => 1
[1, 0] => 1
[1, 1] => 0


このように、パーセプトロンの層を積み重ねることでより複雑な分類問題を解くことが可能である

この「層を積み重ねる」という考えは、ディープラーニングの基本思想の1つであるため、極めて重要と言える

今回のXOR分類問題は、中間層を一つ追加するだけで解くことができたが、より多くの中間層を積み重ねればより複雑な問題にも応用可能となる

![perceptron2](./img/perceptron2.png)